In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gamma
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import *
from sklearn.calibration import calibration_curve, CalibrationDisplay, CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from matplotlib.gridspec import GridSpec

from features import get_feature_columns_for_healthscore
from data import (
    data_dir, project_dir, model_dir
)

from datetime import datetime
import warnings, os, shutil, joblib
warnings.filterwarnings(action='ignore') 

sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(5,5)})
pd.set_option('display.max_columns', None)


In [2]:
## constants
np.random.seed(369)
rs = 369
cp_bins = 30

In [3]:
# plot hist
def plot_prob_dist(y_score, title):
    prob_dist = (1 - y_score) * 1000
    print(pd.DataFrame(prob_dist, columns = ['score']).describe())

    fig, ax = plt.subplots(figsize=(6,6))
    ax = sns.distplot(
        prob_dist, 
        kde=False,
        hist=True,
        bins=cp_bins,
        color='blue',
        # hist_kws={"linewidth": 15,'alpha':1}
        )
    ax.set(
        xlabel='(1-predict_proba)*1000', 
        ylabel='Frequency', 
        title = title, 
        # xlim = (0,1000)
        )
    plt.show()

In [4]:
# calib functions
'''
Attempts to severe imbalance due to low prevalence using various methods
https://towardsdatascience.com/how-to-deal-with-imbalanced-data-in-python-f9b71aba53eb
'''
## CUSTOM FUNCTIONS
# Hosmer-Lemeshow 
def HosmerLemeshow (pihat,Y):
    pihatcat=pd.cut(pihat, np.percentile(pihat,[0,25,50,75,100]),labels=False,include_lowest=True) #here we've chosen only 4 groups

    meanprobs =[0]*4 
    expevents =[0]*4
    obsevents =[0]*4 
    meanprobs2=[0]*4 
    expevents2=[0]*4
    obsevents2=[0]*4 

    for i in range(4):
        meanprobs[i]=np.mean(pihat[pihatcat==i])
        expevents[i]=np.sum(pihatcat==i)*np.array(meanprobs[i])
        obsevents[i]=np.sum(Y[pihatcat==i])
        meanprobs2[i]=np.mean(1-pihat[pihatcat==i])
        expevents2[i]=np.sum(pihatcat==i)*np.array(meanprobs2[i])
        obsevents2[i]=np.sum(1-Y[pihatcat==i]) 


    data1={'meanprobs':meanprobs,'meanprobs2':meanprobs2}
    data2={'expevents':expevents,'expevents2':expevents2}
    data3={'obsevents':obsevents,'obsevents2':obsevents2}
    m=pd.DataFrame(data1)
    e=pd.DataFrame(data2)
    o=pd.DataFrame(data3)

    # The statistic for the test, which follows, under the null hypothesis,
    # The chi-squared distribution with degrees of freedom equal to amount of groups - 2. Thus 4 - 2 = 2
    tt=sum(sum((np.array(o)-np.array(e))**2/np.array(e))) 
    pvalue=1-chi2.cdf(tt,2)

    hl = pd.DataFrame([[chi2.cdf(tt,2).round(2), pvalue.round(2)]],columns = ["Chi2", "p - value"])
    print(hl)

'''
BMR (Bayes Minimum Risk) implementation
Pozzolo et al., 2015, Calibrating Probability with Undersampling
https://towardsdatascience.com/probability-calibration-for-imbalanced-dataset-64af3730eaab
'''
class BMR:
    def beta(binary_target):
        return binary_target.sum()/len(binary_target)
    def tau(binary_target, beta):
        return binary_target.sum()/len(binary_target)
    def calibration(prob, beta):
        return prob/(prob+(1-prob)/beta)

def matrix_metrix(real_values,pred_values,beta, youden=False):
   CM = confusion_matrix(real_values,pred_values)
   TN = CM[0][0]
   FN = CM[1][0] 
   TP = CM[1][1]
   FP = CM[0][1]
   Population = TN+FN+TP+FP
   Prevalence = round( (TP+FN) / Population,4)
   Accuracy   = round( (TP+TN) / Population,4)
   PPV        = round( TP / (TP+FP),4 ) # Precision
   NPV        = round( TN / (TN+FN),4 )
   FDR        = round( FP / (TP+FP),4 )
   FOR        = round( FN / (TN+FN),4 ) 
   check_Pos  = Precision + FDR
   check_Neg  = NPV + FOR
   Recall     = round( TP / (TP+FN),4 )
   FPR        = round( FP / (TN+FP),4 )
   FNR        = round( FN / (TP+FN),4 )
   TNR        = round( TN / (TN+FP),4 ) 
   TPR        = round( TP / (TP+FN),4 )
   check_Pos2 = Recall + FNR
   check_Neg2 = FPR + TNR
   LRPos      = round( Recall/FPR,4 ) 
   LRNeg      = round( FNR / TNR ,4 )
   DOR        = round( LRPos/LRNeg)
   F1         = round ( 2 * ((Precision*Recall)/(Precision+Recall)),4)
   FBeta      = round ( (1+beta**2)*((Precision*Recall)/((beta**2 * Precision)+ Recall)) ,4)
   MCC        = round ( ((TP*TN)-(FP*FN))/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))  ,4)
   BM         = Recall+TNR-1
   MK         = Precision+NPV-1
   mat_met = pd.DataFrame({
'Metric':['TP','TN','FP','FN','Prevalence','Accuracy','PPV','NPV','FDR','FOR','check_Pos','check_Neg','Recall','FPR','FNR','TNR (Specificity)','TPR (Sensitivity)','check_Pos2','check_Neg2','LR+','LR-','DOR','F1','FBeta','MCC','BM','MK'],     
'Value':[TP,TN,FP,FN,Prevalence,Accuracy,PPV,NPV,FDR,FOR,check_Pos,check_Neg,Recall,FPR,FNR,TNR,TPR,check_Pos2,check_Neg2,LRPos,LRNeg,DOR,F1,FBeta,MCC,BM,MK]})
   return (mat_met)



In [5]:
jung_basic_feats = ['AGE', 'TOT_CHOLE', 'LDL_CHOLE', 'HDL_CHOLE', 'TRIGLYCERIDE', 'BLDS', 'HMG', 
    'SGOT_AST', 'SGPT_ALT', 'GAMMA_GTP', 'BMI', 'WAIST', 'BP_HIGH', 'BP_LWST', 'eGFR',
    'SMK_COMBINED', 'DRK_COMBINED', 'METs']

jung_advanced_feats = ['FMLY_APOP_PATIEN_YN', 'FMLY_HPRTS_PATIEN_YN', 
    'FMLY_DIABML_PATIEN_YN', 'FMLY_HDISE_PATIEN_YN']

disc_feats = [
    'SMK_COMBINED',                
    'DRK_COMBINED',               
    'FMLY_APOP_PATIEN_YN',
    'FMLY_HPRTS_PATIEN_YN', 
    'FMLY_DIABML_PATIEN_YN',
    'FMLY_HDISE_PATIEN_YN'] 


In [6]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
og_df.columns

Index(['PERSON_ID', 'SEX', 'AGE', 'BMI', 'WAIST', 'BP_HIGH', 'BP_LWST', 'BLDS',
       'TOT_CHOLE', 'TRIGLYCERIDE', 'HDL_CHOLE', 'LDL_CHOLE', 'HMG',
       'OLIG_PROTE_CD', 'SGOT_AST', 'SGPT_ALT', 'GAMMA_GTP',
       'HCHK_APOP_PMH_YN', 'HCHK_HDISE_PMH_YN', 'HCHK_HPRTS_PMH_YN',
       'HCHK_DIABML_PMH_YN', 'HCHK_HPLPDM_PMH_YN', 'HCHK_ETCDSE_PMH_YN',
       'HCHK_PHSS_PMH_YN', 'FMLY_APOP_PATIEN_YN', 'FMLY_HDISE_PATIEN_YN',
       'FMLY_HPRTS_PATIEN_YN', 'FMLY_DIABML_PATIEN_YN',
       'FMLY_CANCER_PATIEN_YN', 'METs', 'eGFR', 'Death_EVENT', 'MACE_EVENT',
       'Cancer_EVENT', 'SMK_COMBINED', 'DRK_COMBINED'],
      dtype='object')

In [ ]:
og_df.describe().loc[['min', 'max']]

,PERSON_ID,SEX,AGE,BMI,WAIST,BP_HIGH,BP_LWST,BLDS,TOT_CHOLE,TRIGLYCERIDE,HDL_CHOLE,LDL_CHOLE,HMG,OLIG_PROTE_CD,SGOT_AST,SGPT_ALT,GAMMA_GTP,HCHK_APOP_PMH_YN,HCHK_HDISE_PMH_YN,HCHK_HPRTS_PMH_YN,HCHK_DIABML_PMH_YN,HCHK_HPLPDM_PMH_YN,HCHK_ETCDSE_PMH_YN,HCHK_PHSS_PMH_YN,FMLY_APOP_PATIEN_YN,FMLY_HDISE_PATIEN_YN,FMLY_HPRTS_PATIEN_YN,FMLY_DIABML_PATIEN_YN,FMLY_CANCER_PATIEN_YN,METs,eGFR,Death_EVENT,MACE_EVENT,Cancer_EVENT,SMK_COMBINED,DRK_COMBINED
min,10001121.0,0.0,47.0,11.759766,0.0,66.0,30.0,40.0,47.0,3.0,2.0,1.0,3.799805,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,347.0,0.369327,0.0,0.0,0.0,0.0,0.0
max,99999744.0,1.0,89.0,70.046875,129.0,234.0,180.0,785.0,820.0,999.0,954.0,994.0,23.296875,6.0,886.0,989.0,999.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2776.0,259.705872,1.0,1.0,1.0,4.0,420.0


In [ ]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [7]:
# settings

score_types = ['M', 'F']
algorithm = 'logistic'
feat_sets = ['basic', 'plus_advanced']
calib_type = 'uncalib'
scaler = MinMaxScaler()

In [8]:
db = 'NHIS_100'
og_df = pd.read_pickle(
    os.path.join(data_dir, 'processed', 'Healthscore_features_wCancer_{}.pkl'.format(db)))
print(og_df.shape)

(178610, 36)


In [9]:
# model
model = LogisticRegression(
                # penalty='none',
                random_state=rs,
                n_jobs=-1,
                max_iter=2000,
                class_weight='balanced', 
                solver="sag",
                )

In [10]:
outlier_feats = ['TOT_CHOLE', 'LDL_CHOLE', 'HDL_CHOLE', 'TRIGLYCERIDE', 'BLDS', 'HMG', 
    'SGOT_AST', 'SGPT_ALT', 'GAMMA_GTP', 'WAIST', 'BP_HIGH', 'BP_LWST', 'eGFR']

In [11]:
# settings
normalize = True
targets = {
    'mortality':'Death_EVENT',
    'mace':'MACE_EVENT',
    'cancer':'Cancer_EVENT',
    }

df0 = og_df

# remove outliers
before_cnt = df0.shape[0]
print('before removing outliers:', before_cnt)
n_std = 3
for col in outlier_feats:
    col_mean = df0[col].mean()
    col_std = df0[col].std()
    df0 = df0[(df0[col] <= col_mean+(n_std*col_std)) & \
        (df0[col] >= col_mean - (n_std*col_std))] # two tailed
after_cnt = df0.shape[0]
print('after removing outliers:', after_cnt)
print('Rows reduced:', before_cnt - after_cnt)

# run model
for target in targets:
    target_column = targets[target]
    run_name = '{}_{}_{}'.format(db, target, calib_type)
    print('------Model Type:', run_name)
    output_pth = os.path.join('outputs', run_name)
    if os.path.exists(output_pth):
        shutil.rmtree(output_pth)
    os.makedirs(output_pth)
    coef_dfs = []
    minmax_dfs = []
    for feat_set in feat_sets:
        for score_type in ('M', 'F'):
            df = df0
            init_cnt = df.shape[0]
            # print('init row cnt:', init_cnt)
            if score_type=='M':
                df = df[df['SEX']==1]
                np.random.seed(2020)
                pids = df['PERSON_ID'].unique()
                test_pids_man = np.random.choice(pids, size=int(len(pids) * 0.2))
                test_mask_man = np.in1d(df['PERSON_ID'], test_pids_man)
                test_pids = test_pids_man
                test_mask = test_mask_man
            elif score_type=='F':
                df = df[df['SEX']==0]
                np.random.seed(2020)
                pids = df['PERSON_ID'].unique()
                test_pids_woman = np.random.choice(pids, size=int(len(pids) * 0.2))
                test_mask_woman = np.in1d(df['PERSON_ID'], test_pids_woman)
                test_pids = test_pids_woman
                test_mask = test_mask_woman

            # prevalence
            neg, pos = df[target_column].value_counts()
            prevalence = pos / (pos + neg)
            # print('Prevalence for {}: {}'.format(score_type,round(prevalence,4)))
            # print('(pos: {}, neg: {})'.format(pos, neg))

            # create polynomial feature
            multi_col = ['METs']#, 'DRK_COMBINED']
            for col in multi_col:
                df[col+'^2'] = df[col]**2

            if feat_set == 'basic':
                selected_features = jung_basic_feats + ['METs^2']
            else:
                selected_features = jung_basic_feats + ['METs^2'] + jung_advanced_feats 
            print('Using', feat_set, 'feats...')
            # print(selected_features)
                
            print('---Training healthscore model for {}'.format(score_type))
            data = df[selected_features + [target_column]]

            # train test split
            train_df, test_df = data[~test_mask], data[test_mask]

            # normalize
            if normalize:
                if feat_set != 'basic':
                    # get min max
                    minmax_df = data[~test_mask][selected_features].describe().loc[
                        ['min', 'max']].round().T.reset_index(drop=True)
                    minmax_dfs.append(minmax_df)

                # fit, transform
                scaler.fit(train_df[selected_features])
                train_df[selected_features] = scaler.transform(train_df[selected_features])
                test_df[selected_features] = scaler.transform(test_df[selected_features])

            # model
            model.fit(train_df[selected_features], train_df[target_column])
            y_prob = model.predict_proba(test_df[selected_features])[:, 1]
            y_pred = np.array(y_prob > 0.5, dtype=np.int32)
            y_test = np.array(test_df[target_column], dtype=np.int32)

            roc_auc_ = roc_auc_score(y_test, y_prob)
            fpr, tpr, th = roc_curve(y_test, y_prob)
            apr_ = average_precision_score(y_test, y_prob)
            precision, recall, _ = precision_recall_curve(y_test, y_prob)

            # pre youden metrics
            AUC = round(roc_auc_, 4)
            APR = round(apr_, 4)

            CM = confusion_matrix(y_test, y_pred)
            TN = CM[0][0]
            FN = CM[1][0] 
            TP = CM[1][1]
            FP = CM[0][1]
            Population = TN+FN+TP+FP
            Prevalence = round( (TP+FN) / Population,4)
            Precision  = round( TP / (TP+FP),4 ) # PPV
            Recall     = round( TP / (TP+FN),4 )
            Accuracy   = round( (TP+TN) / Population,4)
            F1         = round ( 2 * ((Precision*Recall)/(Precision+Recall)),4)
            
            # post youden metrics
            youden = th[np.argmax(np.abs(tpr - fpr))]
            y_pred = np.array(y_prob > youden, dtype=np.int32)

            CM = confusion_matrix(y_test, y_pred)
            TN = CM[0][0]
            FN = CM[1][0] 
            TP = CM[1][1]
            FP = CM[0][1]
            Population = TN+FN+TP+FP
            # Precision  = round( TP / (TP+FP),4 )
            # Recall     = round( TP / (TP+FN),4 )
   
            PPV        = round( TP / (TP+FP),4 ) # Precision
            NPV        = round( TN / (TN+FN),4 )
            TNR        = round( TN / (TN+FP),4 ) # specificity
            TPR        = round( TP / (TP+FN),4 ) # sensitivity, recall

            # Metrics df
            metrics_df = pd.DataFrame({
                'Metric':['AUC', 'APR', 'Prevalence', 'ACC', 'F1', 'PPV (Precision)', 'NPV', 'TNR (Specificity)', 'TPR (Sensitivity'],
                'Value':[AUC, APR, Prevalence, Accuracy, F1, PPV, NPV, TNR, TPR]
            })
            print(metrics_df)


            # ## Plots
            # # score plots
            # title='Score Distribution for {} (Sex={}, {}, Cnt={})'.format(
            #     target, score_type, feat_set, init_cnt)
            # plot_prob_dist(y_prob, title)

            # # plot auc
            # plt.rcParams['figure.figsize'] = [6,6]
            # plt.plot(fpr, tpr, color='orange', label='AUC = {}'.format(
            #     round(roc_auc_, 4)))
            # plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
            # plt.xlabel('False Positive Rate')
            # plt.ylabel('True Positive Rate')
            # title = "ROC Curve for Logit - {}, {}, {}".format(
            #     target, score_type, feat_set) 
            # plt.title(title)
            # plt.legend()
            # plt.show()

            # # plot apr
            # plt.rcParams['figure.figsize'] = [6,6]
            # plt.plot(recall, precision, color='orange', label='APR = {}'.format(
            #     round(apr_, 4)))
            # plt.plot([0, 1], [0, 0], color='darkblue', linestyle='--')
            # plt.xlabel('Recall')
            # plt.ylabel('Precision')
            # title = "Precision Recall Curve for Logit - {}, {}, {}".format(
            #     target, score_type, feat_set) 
            # plt.title(title)
            # plt.legend()
            # plt.show()

            # # plot METs
            # sns.regplot(x=data['METs']*1000, y=data[target_column],  
            #     logistic=False, ci=None, scatter=False, color='orange', label='Linear')
            # sns.regplot(x=data['METs^2']*1000, y=data[target_column]*10, 
            #     order=2, ci=None, scatter=False, color='blue', label='Polynomial')
            # plt.xlabel('METs')
            # plt.ylabel('Death-logit')
            # title = "METs Feature Analysis - {}, {}, {}".format(target, score_type, feat_set) 
            # plt.title(title)
            # plt.legend()
            # plt.show()

            # # plot SMK_COMBINED
            # sns.regplot(x=data['SMK_COMBINED'], y=data[target_column],  
            #     logistic=False, ci=None, scatter=False, color='orange', label='Linear')
            # plt.xlabel('Smoking')
            # plt.ylabel('Death-logit')
            # title = "Smoking Feature Analysis - {}, {}, {}".format(target, score_type, feat_set) 
            # plt.title(title)
            # plt.legend()
            # plt.show()

            # # plot DRK_COMBINED
            # sns.regplot(x=data['DRK_COMBINED'], y=data[target_column],  
            #     logistic=False, ci=None, scatter=False, color='orange', label='Linear')
            # # sns.regplot(x=data['DRK_COMBINED^2'], y=data[target_column], 
            # #     order=2, ci=None, scatter=False, color='blue', label='Polynomial')
            # plt.xlabel('Drinking')
            # plt.ylabel('Death-logit')
            # title = "Drinking Feature Analysis - {}, {}, {}".format(target, score_type, feat_set) 
            # plt.title(title)
            # plt.legend()
            # plt.show()

            # # coefs
            # intercept = [['_intercept', model.intercept_[0]]]
            # rows = [[f, c] for f, c in zip(selected_features, model.coef_[0])]
            # rows = intercept + rows
            # header = ['{}_{}_features'.format(score_type, feat_set), 'coefficients']
            # coef_df = pd.DataFrame(rows, columns=header)
            # coef_dfs.append(coef_df)

        # # Set Calibration Plot
        # fig = plt.figure(figsize=(10, 10))
        # plt.rcParams['savefig.facecolor']='white'
        # plt.rcParams['savefig.edgecolor']='none'
        # gs = GridSpec(4, 2)
        # colors = plt.cm.get_cmap("Dark2")
        # ax_calibration_curve = fig.add_subplot(gs[:2, :2])
        # calibration_displays = {}
        
        # # Add to calibration plot
        # print('Adding to CalibrationPlot...')
        # display = CalibrationDisplay.from_predictions(
        #     y_test,
        #     y_prob,
        #     n_bins=cp_bins,
        #     name=algorithm,
        #     ax=ax_calibration_curve,
        #     color=colors(0)
        # )
        # calibration_displays[0] = display

        # # Finalize calibration plot
        # ax_calibration_curve.grid()
        # ax_calibration_curve.set_title(
        #     "Calibration Plots - {}, {}- Sex={}, FeatSet={}, n_bins={})".format(
        #     target, calib_type, score_type, feat_set, cp_bins) 
        # )
        # plt.show()

        print()
    
    # save coef_dfs
    fpath = os.path.join(project_dir, 'reports', 
        'coeffs_JUNG_logit_{}_{}.csv'.format(target, db))
    pd.concat(coef_dfs + minmax_dfs, axis=1).to_csv(fpath, index=False)

    print()


before removing outliers: 178610
after removing outliers: 154294
Rows reduced: 24316
------Model Type: NHIS_100_mortality_uncalib
Using basic feats...
---Training healthscore model for M
              Metric   Value
0                AUC  0.8088
1                APR  0.1873
2         Prevalence  0.0354
3                ACC  0.4670
4                 F1  0.1063
5    PPV (Precision)  0.1145
6                NPV  0.9861
7  TNR (Specificity)  0.8043
8   TPR (Sensitivity  0.6903
Using basic feats...
---Training healthscore model for F
              Metric   Value
0                AUC  0.7784
1                APR  0.1395
2         Prevalence  0.0184
3                ACC  0.9758
4                 F1  0.1934
5    PPV (Precision)  0.0748
6                NPV  0.9916
7  TNR (Specificity)  0.8577
8   TPR (Sensitivity  0.6126

Using plus_advanced feats...
---Training healthscore model for M
              Metric   Value
0                AUC  0.8179
1                APR  0.2088
2         Prevalence  0

In [12]:
data[~test_mask][selected_features].describe().loc[
                        ['min', 'max']].round().T

,min,max
AGE,47.0,88.0
TOT_CHOLE,89.0,311.0
LDL_CHOLE,7.0,229.0
HDL_CHOLE,11.0,137.0
TRIGLYCERIDE,3.0,409.0
BLDS,40.0,175.0
HMG,10.0,18.0
SGOT_AST,2.0,72.0
SGPT_ALT,1.0,64.0
GAMMA_GTP,1.0,149.0


In [13]:
og_df.describe().loc[['min', 'max']].round().T

,min,max
PERSON_ID,10001121.0,99999744.0
SEX,0.0,1.0
AGE,47.0,89.0
BMI,12.0,70.0
WAIST,0.0,129.0
BP_HIGH,66.0,234.0
BP_LWST,30.0,180.0
BLDS,40.0,785.0
TOT_CHOLE,47.0,820.0
TRIGLYCERIDE,3.0,999.0
